In [2]:
from product_descriptor.run import run
run("")

In [5]:
from product_descriptor.processing.post_processing.extract import extract_transcript,expand_information,get_product_info

In [7]:
import os
os.environ['h'] = 'hi'

In [9]:
os.environ['h']

'hi'

In [20]:
data = extract_transcript("K5B2y_4Tu_k")
info = get_product_info(data)

In [ ]:
print(expand_information(info))

In [1]:
path_sample1 = "product_descriptor/processing/post_processing/sample.txt"
path_sample2 = "product_descriptor/processing/post_processing/sample2.txt"

with open(path_sample1,"r") as f:
    report1 = f.read()
with open(path_sample2,"r") as f:
    report2 = f.read()

In [2]:
product_comparision_prompt = """You are an AI assistant specialized in generating product comparision sheets.
You will get summarized review of two products and you have to neatly draft a tabular comparision sheet with pointers comparing the two products.
The individual product summary sheets will contain the following detials:
- pros
- cons
- specifications
- key insights
- reviewer opinion
You have to compare them on the bases of these fields. Do a direct comparision and compare similar features and tell which is better in the tabular sheet.
Do mention if any product has a feature that the other misses or have a specification that on paper is better than the other mention it in the comparision. For ex: Product a has s pen included, product b does not have a s pen
Output Format:
heading : which aspect of the product is being compared. Ex design, processesor, camera etc
sidea : pointer for product a
sideb : pointer for product b 
summary : a summary for the comparision which is better side a or b with reason. 
You must output a list containing each row for the comparision in the above format
You must give each unique feature its own header. Do not concatinate two features/differences in one row.
"""
from dotenv import load_dotenv
from pydantic import BaseModel
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=st.secrets['openai'])

class comparision(BaseModel):
    heading : str
    sidea : str
    sideb : str
    summary : str
    
class table(BaseModel):
    rows : list[comparision]
    

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "system", "content": product_comparision_prompt},
        {"role": "user", "content": f"PRODUCT 1 :\n{report1} \n\n\nPRODUCT 2 :\n{report2}"},
    ],
    response_format = table
)
product_info = completion.choices[0].message.parsed

In [ ]:
for i in product_info.rows:
    print(i.heading)
    print(i.sidea)
    print(i.sideb)
    print(i.summary)
    print()

In [1]:
from product_descriptor.rag.chunk_handler import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.txt_loader(id = "K5B2y_4Tu_k")

In [ ]:
chunks

In [1]:
from product_descriptor.rag.chatbot import create_embeddings
embeddings = create_embeddings("K5B2y_4Tu_k")

In [2]:
embeddings

In [ ]:
from product_descriptor.rag.chatbot import ask_question
answer = ask_question("How is the design of new dell laptop",ids = ["K5B2y_4Tu_k"])


In [ ]:
answer.choices[0].message.content